# Set up and train

Upload:.yaml

Attention: make sure the index matches the classes

In [ ]:
!nvidia-smi
!pip -q install ultralytics opencv-python pyyaml
from ultralytics import settings, YOLO
settings.update({'datasets_dir': '/content/datasets'})
!yolo checks
from google.colab import drive
drive.mount('/content/drive')

Tue Dec  2 03:26:16 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Yolov8 formal train(get "best.pt")

In [ ]:
import time, os, glob, shutil

EXP = f"ppe_v8s_img960_e100_{time.strftime('%m%d-%H%M')}"
PROJECT = "/content/runs"   #
RUN_DIR = f"{PROJECT}/{EXP}"

!yolo detect train \
    model=yolov8s.pt \
    data=construction-ppe.yaml \
    imgsz=960 batch=16 epochs=100 workers=4 device=0 patience=30 \
    classes=0,2,6 \
    project={PROJECT} name={EXP}




Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=(0, 2, 6), close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=construction-ppe.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=ppe_v8s_img960_e100_1118-2310, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=30, perspective=0.0,

Zip results and save to drive

In [ ]:
zip_base = "/content/runs_backup"
if os.path.exists(zip_base + ".zip"):
    os.remove(zip_base + ".zip")
shutil.make_archive(zip_base, "zip", RUN_DIR)  # /content/runs_backup.zip
print("ZIP created:", zip_base + ".zip")

#Save to Drive
ts = time.strftime("%m%d_%H%M")
save_dir = "/content/drive/MyDrive/ppe_runs"
os.makedirs(save_dir, exist_ok=True)
dst = f"{save_dir}/ppe_with_factory_data_026{ts}.zip"
shutil.move(zip_base + ".zip", dst)
print("✅ Copied to:", dst)

ZIP created: /content/runs_backup.zip
✅ Copied to: /content/drive/MyDrive/ppe_runs/ppe_with_factory_data_0261119_0051.zip


# Test the test set images under different confidence

In [ ]:
from ultralytics import YOLO
from math import isfinite

MODEL = "/content//best.pt"
DATA  = "/content/construction-ppe.yaml"
PROJECT = "/content/drive/MyDrive/yolo_eval_outputs"
IMGSZ = 960
CLASSES = [0, 2, 6]  # person/vest/helmet

model = YOLO(MODEL)
#test with different confidence
def one_split(split, conf_list=(0.25, 0.50, 0.75)):
    # draw images
    model.val(
        data=DATA, split=split, imgsz=IMGSZ,
        iou=0.5, conf=0.001, plots=True, classes=CLASSES,
        project=PROJECT, name=f"{split}_curves", verbose=False
    )

    for t in conf_list:
        m = model.val(
            data=DATA, split=split, imgsz=IMGSZ,
            iou=0.5, conf=t, plots=False, classes=CLASSES,
            project=PROJECT, name=f"{split}_conf_{str(t).replace('.','')}",
            verbose=False
        )
        P = m.results_dict.get('metrics/precision(B)', 0.0) or 0.0
        R = m.results_dict.get('metrics/recall(B)', 0.0) or 0.0
        F1 = (2*P*R/(P+R)) if (P+R) > 0 else 0.0
        map50   = m.results_dict.get('metrics/mAP50(B)', 0.0) or 0.0
        map5095 = m.results_dict.get('metrics/mAP50-95(B)', 0.0) or 0.0
        print(f"[{split} @ conf={t:.2f}]  P={P:.3f}  R={R:.3f}  F1={F1:.3f}  mAP@0.5={map50:.3f}  mAP@0.5:0.95={map5095:.3f}")


one_split("test")


Ultralytics 8.3.232 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,129,841 parameters, 0 gradients, 28.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2133.5±238.6 MB/s, size: 79.6 KB)
val: Scanning /content/datasets/construction-ppe-final6/labels/test.cache... 213 images, 8 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 213/213 359.8Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 14/14 1.7it/s 8.2s
                   all        213        995      0.914      0.878      0.924      0.586
Speed: 8.1ms preprocess, 18.3ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to /content/drive/MyDrive/yolo_eval_outputs/test_curves2
Ultralytics 8.3.232 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2904.4±1032.0 MB/s, size: 85.9 KB)
val: Scanning /content/datasets/construction-ppe-final6/

# Track with person ID in videos

upload:

1.Using the test videos(mp4) in google drive

2."best.pt"





In [ ]:
import cv2, numpy as np, os, time, shutil

def mount_drive_once(mount_point="/content/drive"):
    if not os.path.ismount(mount_point):
        drive.mount(mount_point)

def person_index_from_names(names: dict) -> int:
    for k, v in names.items():
        if str(v).lower() == "person" or "person" in str(v).lower():
            return int(k)
    return 0

def track_person_draw_all(
    model_path: str,
    source_path: str,
    drive_out_dir: str = "/content/drive/MyDrive/ppe_track",
    save_name: str | None = None,
    conf: float = 0.40,
    iou: float = 0.50,
    imgsz: int = 960,
    tracker_cfg: str = "bytetrack.yaml",
    draw_colors=((0,255,0), (255,255,255)),  # (person color, ppe color)
):
    """
    Detect all classes; track via model.track (ByteTrack); render ID only for 'person'.
    Save final mp4 to Google Drive.
    """
    mount_drive_once()

    # --- Load model & resolve class indices ---
    model = YOLO(model_path)
    names = model.names
    if isinstance(names, dict):
        id2name = [names[i] for i in sorted(names.keys())]
    else:
        id2name = list(names)
    def find_idx(targets):
        targets = [t.lower() for t in targets]
        for i, n in enumerate(id2name):
            if str(n).lower() in targets:
                return i
        return None

    pid  = find_idx(["person"])
    vid  = find_idx(["vest"])
    hid  = find_idx(["helmet"])
    #pid = person_index_from_names(names)

    # --- Prepare video IO (read first frame to get size/fps) ---
    cap = cv2.VideoCapture(source_path)
    assert cap.isOpened(), f"Cannot open video: {source_path}"
    fps = cap.get(cv2.CAP_PROP_FPS) or 30
    W  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()

    ts = time.strftime("%m%d_%H%M")
    base = os.path.splitext(os.path.basename(source_path))[0]
    if save_name is None:
        save_name = f"{base}_personID_{ts}.mp4"

    local_tmp_dir = "/content/_ppe_out"
    os.makedirs(local_tmp_dir, exist_ok=True)
    local_out = os.path.join(local_tmp_dir, save_name)
    writer = cv2.VideoWriter(local_out, cv2.VideoWriter_fourcc(*"mp4v"), fps, (W, H))

    p_color, o_color = draw_colors
    vest_color = (0, 0, 255)

    print("[INFO] Running detection+tracking (ByteTrack via model.track)...")
    # persist=True keeps tracks across frames in stream mode
    for res in model.track(
        source=source_path,
        stream=True,
        conf=conf, iou=iou, imgsz=imgsz,
        tracker=tracker_cfg,
        persist=True,
        verbose=False,
    ):
        frame = res.orig_img  # BGR

        if res.boxes is None or len(res.boxes) == 0:
            writer.write(frame)
            continue

        boxes = res.boxes
        xyxy = boxes.xyxy.cpu().numpy()
        cls  = boxes.cls.cpu().numpy().astype(int)
        scr  = boxes.conf.cpu().numpy()
        ids  = boxes.id
        ids  = ids.cpu().numpy().astype(int) if ids is not None else None

        # draw
        for i, ((x1,y1,x2,y2), k, s) in enumerate(zip(xyxy, cls, scr)):
            x1, y1, x2, y2 = map(int, (x1,y1,x2,y2))
            if k == pid and ids is not None:
                tid = int(ids[i])
                cv2.rectangle(frame, (x1,y1), (x2,y2), p_color, 2)
                cv2.putText(frame, f"person #{tid}", (x1, max(0,y1-6)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, p_color, 2)
            else:
                label = id2name[k]

                if vid is not None and k == vid:
                    color = vest_color

                elif hid is not None and k == hid:
                    color = (255, 0, 0) # helmet

                else:
                    color = o_color
                #draw rectangle
                cv2.rectangle(frame, (x1,y1), (x2,y2), color, 2)

                cv2.putText(frame, label, (x1, max(0,y1-6)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        writer.write(frame)

    writer.release()

    # --- Move to Google Drive ---
    os.makedirs(drive_out_dir, exist_ok=True)
    dst = os.path.join(drive_out_dir, save_name)
    shutil.move(local_out, dst)
    print("[DONE] Saved to:", dst)
    return dst


In [ ]:
out_path = track_person_draw_all(
    model_path="/content/best.pt",
    source_path="/content/drive/MyDrive/ppe_videos/11_4_4.mp4",
    drive_out_dir="/content/drive/MyDrive/ppe_track",
    conf=0.40, iou=0.50, imgsz=960
)
print(out_path)


[INFO] Running detection+tracking (ByteTrack via model.track)...
requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 2 packages in 146ms
Prepared 1 package in 43ms
Installed 1 package in 1ms
 + lap==0.5.12

requirements: AutoUpdate success ✅ 0.6s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect

[DONE] Saved to: /content/drive/MyDrive/ppe_track/11_4_4_personID_1125_1700.mp4
/content/drive/MyDrive/ppe_track/11_4_4_personID_1125_1700.mp4


save as json

In [ ]:
# # --- Extra cell: export tracking results to JSON ---
# import json
# from ultralytics import YOLO

# def track_to_json(
#     model_path: str,
#     source_path: str,
#     json_out_path: str = "/content/drive/MyDrive/ppe_track/sample_detection.json",
#     conf: float = 0.40,
#     iou: float = 0.50,
#     imgsz: int = 960,
#     tracker_cfg: str = "bytetrack.yaml",
#     max_frames: int | None = 50,  # only export first N frames for sample
# ):
#     """
#     Run model.track and export detection+tracking results to a JSON file.

#     JSON format: a flat list, each item is
#     {
#       "frame_index": int,
#       "track_id": int or null,
#       "class_id": int,
#       "class_name": str,
#       "conf": float,
#       "bbox_xyxy": [x1, y1, x2, y2]
#     }
#     """
#     mount_drive_once()  # use the function you already defined

#     # Load model and build id2name list
#     model = YOLO("/content/best.pt")
#     names = model.names
#     if isinstance(names, dict):
#         id2name = [names[i] for i in sorted(names.keys())]
#     else:
#         id2name = list(names)

#     all_records = []
#     frame_idx = -1

#     print("[INFO] Running model.track for JSON export...")
#     for res in


# Predict frames

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
from ultralytics import YOLO

MODEL_PATH = "/content/best.pt"
IMAGE_SOURCE_PATH = "/content/drive/MyDrive/ppe_frames/frames_out/11_4_2"

# check if the file exist
if not os.path.exists(MODEL_PATH):
    print(f"❌ Cannot find the file {MODEL_PATH}")
elif not os.path.exists(IMAGE_SOURCE_PATH):
    print(f"❌ Cannot find the image folder {IMAGE_SOURCE_PATH}")
else:
    print("✅ Begin to predict.")
    !yolo predict model={MODEL_PATH} \
                    source={IMAGE_SOURCE_PATH} \
                    project="yolo_runs" \
                    name="predict_results" \
                    imgsz=960 conf=0.40 iou=0.5\
                    save_txt=True \
                    save_conf=True\
                    save=False

In [ ]:
import shutil

OUT_ROOT = "/content/yolo_runs/predict_results"

IMG_DST = os.path.join(OUT_ROOT, "images")
os.makedirs(IMG_DST, exist_ok=True)

img_exts = (".jpg", ".jpeg", ".png")

count = 0
for fname in os.listdir(IMAGE_SOURCE_PATH):
    if fname.lower().endswith(img_exts):
        src = os.path.join(IMAGE_SOURCE_PATH, fname)
        dst = os.path.join(IMG_DST, fname)
        shutil.copy2(src, dst)
        count += 1

print(f"✅ Copied {count} images to {IMG_DST}")


✅ Copied 31 images to /content/yolo_runs/predict_results/images


Create null label files

In [ ]:
from pathlib import Path
LABELS_DIR = Path("yolo_runs/predict_results/labels")
LABELS_DIR.mkdir(parents=True, exist_ok=True)

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}

total_imgs, created_empties, existed = 0, 0, 0

for p in sorted(IMAGE_SOURCE_PATH.rglob("*")):
    if p.suffix.lower() not in IMG_EXTS:
        continue
    total_imgs += 1
    txt_path = LABELS_DIR / f"{p.stem}.txt"
    if txt_path.exists():
        existed += 1
    else:
        # create null txt.
        txt_path.touch()
        created_empties += 1

print(f"[DONE] Find {total_imgs} images")
print(f"      Annotated {existed} images")
print(f"      Created {created_empties} null annotation txt.")
print(f"label content:{LABELS_DIR.resolve()}")


Transform the label txt. to json.

In [ ]:
import os, json, glob

LABEL_DIR = "yolo_runs/predict_results/labels"
JSON_OUT = "yolo_runs/predict_results/labels_102403.json"

records = []

for txt_path in glob.glob(os.path.join(LABEL_DIR, "*.txt")):
    img_name = os.path.basename(txt_path).replace(".txt", ".jpg")
    with open(txt_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) not in (5, 6):
                continue
            cls = int(parts[0])
            xc, yc, w, h = map(float, parts[1:5])
            conf = float(parts[5]) if len(parts) == 6 else None

            rec = {
                "image_name": img_name,
                "class_id": cls,
                "x_center": xc,
                "y_center": yc,
                "width": w,
                "height": h,
                "conf": conf,
            }
            records.append(rec)

with open(JSON_OUT, "w", encoding="utf-8") as f:
    json.dump(records, f, indent=2)

print("JSON saved to:", JSON_OUT)
print("Total records:", len(records))


JSON saved to: yolo_runs/predict_results/labels_102403.json
Total records: 254


# Convert videos into frames

In [ ]:
import cv2
from pathlib import Path
from google.colab import drive

drive.mount('/content/drive')
video_path = Path("/content/drive/MyDrive/ppe_videos/11_4_2.mp4")
frames_dir = Path("/content/drive/MyDrive/ppe_frames/frames_out/11_4_2")

# Save an image every N frames
FRAME_EVERY = 50  # 1 = every frame, 5 = save 1 frame out of 5
RESIZE_TO = None

# Create output directory if not exists
frames_dir.mkdir(parents=True, exist_ok=True)

# Check video exists
if not video_path.exists():
    raise FileNotFoundError(f"Video not found: {video_path}")

print(f"[INFO] Processing video: {video_path.name}")

cap = cv2.VideoCapture(str(video_path))
if not cap.isOpened():
    raise RuntimeError(f"Failed to open video: {video_path}")

video_name = video_path.stem  # "11_4_2"
frame_idx = 0   # index in original video
saved_idx = 0   # how many frames have been saved

while True:
    ret, frame = cap.read()
    if not ret:
        break  # no more frames

    # Only save every N-th frame
    if frame_idx % FRAME_EVERY == 0:
        if RESIZE_TO is not None:
            # RESIZE_TO: (width, height)
            frame = cv2.resize(frame, RESIZE_TO)

        saved_idx += 1
        # e.g. 11_4_2_000001.jpg
        frame_name = f"{video_name}_{saved_idx:06d}.jpg"
        out_path = frames_dir / frame_name

        cv2.imwrite(str(out_path), frame)

    frame_idx += 1

cap.release()
print(f"[INFO] Done: {video_path.name}, total saved frames = {saved_idx}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[INFO] Processing video: 11_4_2.mp4
[INFO] Done: 11_4_2.mp4, total saved frames = 31
